# Case Cognita - Pipeline Modelo ML

Neste notebook, iremos realizar a importação, limpeza e pré-processamento dos dados do dataset e o treinamento, avaliação e escolha do modelo de machine learning com melhor performance na previsão de default dos clientes da empresa X-Health. Como a análise exploratória dos dados foi realizada em um notebook separado, essa etapa não será incluída neste notebook

## Importação

In [4]:
# Importando bibliotecas necessárias
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, cross_val_score, KFold
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score
from sklearn.impute import SimpleImputer
from imblearn.combine import SMOTEENN
import pickle

In [5]:
# Importando dataframe
defaults_df = pd.read_csv('dataset_2021-5-26-10-14.csv', sep='\t', encoding='utf-8')
defaults_df.head()

,default_3months,ioi_36months,ioi_3months,valor_por_vencer,valor_vencido,valor_quitado,quant_protestos,valor_protestos,quant_acao_judicial,acao_judicial_valor,...,dividas_vencidas_qtd,falencia_concordata_qtd,tipo_sociedade,opcao_tributaria,atividade_principal,forma_pagamento,valor_total_pedido,month,year,default
0,0,58.000000,18.236092,0.00,0.0,242100.70,0,0.0,0,0.0,...,0,0,empresario (individual),simples nacional,papelaria,30/60/90,34665.674938,6,2019,0
1,1,16.052632,7.500000,224132.85,0.0,4960846.21,0,0.0,0,0.0,...,0,0,sociedade empresaria limitada,missing,com de equipamentos de informatica,30/60/90,7134.489373,10,2018,0
2,0,13.250000,3.904762,513043.83,0.0,158631.93,1,1800.0,0,0.0,...,0,0,sociedade empresaria limitada,simples nacional,servicos de vigilancia e guarda,missing,72653.621143,4,2018,0
3,0,136.925000,10.144219,23273.64,0.0,669644.16,0,0.0,0,0.0,...,0,0,empresario (individual),simples nacional,com de equipamentos de informatica,missing,14576.805783,4,2017,1
4,0,140.333333,17.651678,0.00,0.0,2010.56,0,0.0,0,0.0,...,0,0,sociedade empresaria limitada,simples nacional,com de compon eletron e pecas para eletrod,30/60/90,2655.505663,10,2017,0


## Limpeza

Pela análise exploratória realizada anteriormente, identificamos a existência de valores ausentes nas seguintes colunas:
- tipo_sociedade
- opcao_tributaria
- atividade_principal
- forma_pagamento
- participacao_falencia_valor

Como todas essas colunas são features qualitativas (exceto a participacao_falencia_valor) com uma quantidade grande de categorias em cada uma (além de serem as únicas colunas com valores ausentes), por uma questão de maior simplicidade do modelo e melhor eficiência computacional e capacidade de generalização, iremos retirá-las do dataset de treinamento. 

Iremos também excluir a coluna participacao_falencia_valor por estar **completamente** vazia!

In [8]:
defaults_df = defaults_df.drop(['tipo_sociedade', 'opcao_tributaria', 'atividade_principal', 'forma_pagamento', 'participacao_falencia_valor'], axis=1)

Também identificamos nessa etapa uma alta correlação entre as variáveis quant_acao_judicial e acao_judicial_valor, o que sugere uma redundância entre essas duas features. Optamos por manter apenas a quant_acao_judicial, pois tende a apresentar valores menores e, portanto, não geraria tanto viés durante o treinamento ao não ser atribuído um peso excessivamente grande a essa variável

In [10]:
defaults_df = defaults_df.drop('acao_judicial_valor', axis=1)

## Pré-processamento

In [12]:
# Separando features de labels
X = defaults_df.drop('default', axis=1)
y = defaults_df['default'].values

In [13]:
# Criando dataset de treinamento e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=123, stratify=y)

Como identificado na análise exploratória, o dataset de treinamento é **desbalanceado** no que se refere à **proporção de classes** dos labels. Para endereçar esse ponto e evitarmos que o modelo tenha um viés ineficaz, prevendo mais pedidos genuínos do que o ideal, utilizaremos a biblioteca imbalanced-learn (importada como imblearn) e uma combinação de undersampling da classe majoritária e oversampling da classe minoritária

In [15]:
# Aplicando under e oversampling para balanceamento dos labels
smote_enn = SMOTEENN(random_state=123)
X_train, y_train = smote_enn.fit_resample(X_train, y_train)

In [16]:
# Aplicando Standardization nos dados para uniformização
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

## Comparando e escolhendo o melhor modelo de machine learning usando validação cruzada

Os dois modelos de machine learning que serão testados são **logistic regression** e **random forest**. Dentre as razões pelas quais esses modelos foram escolhidos, destacam-se:
- os diferentes tipos de relação que cada modelo consegue captar de forma mais eficiente (relações lineares e não lineares, respectivamente)
- os diferentes níveis de complexidade de cada modelo, sendo o modelo de regressão logística mais simples, porém, mais computacionalmente eficiente
- a presença de parâmetros, durante o processo de treinamento, que podem ser definidos de modo a diminuir o viés do modelo em caso de um dataset de treinamento desbalanceado

In [19]:
# Realizando validação cruzada de 5 folds
# Criando instância KFold
kf = KFold(n_splits=5, shuffle=True, random_state=123)

# Criando instâncias de cada modelo
log_reg = LogisticRegression(random_state=123, class_weight='balanced') # usamos balanced para evitar o viés causado pelo desbalanceamento dos labels
random_forest = RandomForestClassifier(random_state=123, class_weight='balanced')

# Calculando a média do f1-score (métrica de performance escolhida) de cada modelo
cross_val_score_log_reg = cross_val_score(log_reg, X_train, y_train, cv=kf, scoring='f1')
cross_val_score_random_forest = cross_val_score(random_forest, X_train, y_train, cv=kf, scoring='f1')
print(cross_val_score_log_reg.mean(), cross_val_score_random_forest.mean())

0.6436101378784146 0.9442763084646412


Percebemos, assim, que o modelo de random forest tem um desempenho significativamente superior ao do modelo de regressão logística, por isso, escolheremos o primeiro como modelo final

## Treinando o modelo escolhido e avaliando sua performance "final"

In [22]:
# Criando instância do modelo
random_forest = RandomForestClassifier(random_state=456, class_weight='balanced')

# Treinando o modelo com o dataset de treinamento
random_forest.fit(X_train, y_train)

# Avaliando a performance do modelo treinado usando a métrica f1-score
predictions = random_forest.predict(X_test)
f1_score = f1_score(y_test, predictions)
f1_score

0.6723467308760882

## Salvando o modelo para ser utilizado externamente

In [31]:
with open('model.pkl', 'wb') as context:
    pickle.dump(random_forest, context)

Para que o modelo funcione adequadamente mesmo que não sejam fornecidos valores para todas as features, será necessária a criação de um imputer a ser utilizado nesses casos

In [33]:
X_df = defaults_df.drop('default', axis=1)
imputer = SimpleImputer(strategy='median')
imputer.fit(X_df)

SimpleImputer(strategy='median')

In [35]:
with open('imputer.pkl', 'wb') as context:
    pickle.dump(imputer, context)